In [9]:
import sys

In [6]:
try:
  !rm -rf boptestGymService
except:
  pass
!git clone -b boptest-gym-service https://github.com/ibpsa/project1-boptest-gym.git boptestGymService

Cloning into 'boptestGymService'...
remote: Enumerating objects: 4015, done.
remote: Counting objects: 100% (569/569), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 4015 (delta 340), reused 542 (delta 332), pack-reused 3446 (from 1)
Receiving objects: 100% (4015/4015), 49.69 MiB | 6.17 MiB/s, done.
Resolving deltas: 100% (2134/2134), done.


In [10]:
sys.path.insert(0,'boptestGymService')

In [11]:
import gym
import pandas as pd
import matplotlib.pyplot as plt
import torch
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback
from boptestGymEnv import BoptestGymEnv, NormalizedObservationWrapper

In [12]:
# Check if MPS is available
# NB! Will only work if you have a M1 Mac w/ a PyTorch version with MPS support. 
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')

In [13]:
# for reproducibility
seed = 1337

In [27]:
url = 'http://172.18.0.2:5001'


#### PPO training loop


In [28]:
# Initialize the environment
env = BoptestGymEnv(
    url=url,
    testcase='bestest_hydronic_heat_pump',
    actions=['oveHeaPumY_u'],
    observations={
        'time': (0, 604800),
        'reaTZon_y': (280., 310.),
        'PriceElectricPowerHighlyDynamic': (-0.4, 0.4),
        'LowerSetp[1]': (280., 310.),
        'UpperSetp[1]': (280., 310.)
    },
    predictive_period=24*3600,
    regressive_period=6*3600,
    random_start_time=True,
    max_episode_length=24*3600,
    warmup_period=24*3600,
    step_period=3600
)

# Normalize observations
env = NormalizedObservationWrapper(env)

# Initialize the RL agent using PPO
model = PPO('MlpPolicy', env, verbose=1, device=device)

# Train the model
model.learn(total_timesteps=1)

ConnectTimeout: HTTPConnectionPool(host='172.18.0.2', port=5001): Max retries exceeded with url: /testcases/bestest_hydronic_heat_pump/select (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x31d136980>, 'Connection to 172.18.0.2 timed out. (connect timeout=None)'))